In [1]:
import random

In [2]:
CARDS = ['A', 'K', 'Q']
CARD_RANK = {'A': 3, 'K': 2, 'Q': 1}
ANTE = 1
BET = 1

In [3]:
def prob(num, den, op1, op2):
    return op1 if random.randint(1,den) <= num else op2

In [4]:
def showdown(hand1, hand2):
    return 1 if CARD_RANK[hand1] > CARD_RANK[hand2] else 2

In [5]:
# STRATEGY FUNCTIONS (Edit these per your strategy)
# Each function takes hand (str), history (list), and returns an action ('check', 'bet', 'call', 'fold')
def player1_strategy(hand, history):
    # Example: Bet with A, check with K or Q
    if not history:
        if hand == 'A':
            return 'bet'
        elif hand == 'K':
            return 'check'
        else:
            return prob(1,3,'bet','check')
    else:
        if hand == 'K':
            return prob(2,3,'call','fold')
        else:
            return 'fold'
        

In [6]:
def player2_strategy(hand, history):
    # history: list of actions so far. E.g., ["check"] or ["bet"]
    if history == ['bet']:
        if hand == 'A':
            return 'call'
        elif hand == 'K':
            return prob(1,3,'call','fold')
        else:
            return 'fold'
    elif history == ['check']:
        if hand == 'A':
            return 'bet'
        elif hand == 'K':
            return 'check'
        else:
            return prob(1,3,'bet','check')
    else:
        return 'check'

In [7]:
def simulate_akq_hand():
    deck = CARDS[:]
    random.shuffle(deck)
    hand1, hand2 = deck[0], deck[1]
    p1_contribution = ANTE
    p2_contribution = ANTE
    pot = p1_contribution + p2_contribution
    history = []

    # Player 1 acts
    act1 = player1_strategy(hand1, [])
    if act1 == 'bet':
        pot += BET
        p1_contribution += BET
        history.append('bet')
        # Player 2 responds
        act2 = player2_strategy(hand2, history)
        if act2 == 'call':
            pot += BET
            p2_contribution += BET
            winner = showdown(hand1, hand2)
        elif act2 == 'fold':
            winner = 1
        else:
            raise ValueError("Invalid action by Player 2")
    elif act1 == 'check':
        history.append('check')
        # Player 2 acts
        act2 = player2_strategy(hand2, history)
        if act2 == 'check':
            winner = showdown(hand1, hand2)
        elif act2 == 'bet':
            pot += BET
            p2_contribution += BET
            history.append('bet')
            # Player 1 responds
            act3 = player1_strategy(hand1, history)
            if act3 == 'call':
                pot += BET
                p1_contribution += BET
                winner = showdown(hand1, hand2)
            elif act3 == 'fold':
                winner = 2
            else:
                raise ValueError("Invalid action by Player 1")
        else:
            raise ValueError("Invalid action by Player 2")
    else:
        raise ValueError("Invalid action by Player 1")

    # Payoff for each player: what they win minus what they contributed
    if winner == 1:
        payoff_p1 = pot - p1_contribution
        payoff_p2 = -p2_contribution
    else:
        payoff_p1 = -p1_contribution
        payoff_p2 = pot - p2_contribution
    return payoff_p1, payoff_p2


In [8]:
def run_simulation(n_rounds):
    p1_total = 0
    p2_total = 0
    for _ in range(n_rounds):
        payoff1, payoff2 = simulate_akq_hand()
        p1_total += payoff1
        p2_total += payoff2
    print(f"After {n_rounds} rounds:")
    print(f"Player 1 Expected Value per Game: {p1_total / n_rounds:.3f}")
    print(f"Player 2 Expected Value per Game: {p2_total / n_rounds:.3f}")

run_simulation(100000000) 

After 100000000 rounds:
Player 1 Expected Value per Game: -0.056
Player 2 Expected Value per Game: 0.056
